In [3]:
import pandas as pd
import numpy as np

In [4]:
df_address = pd.read_csv('address(in).csv')
df_main = pd.read_csv('main(in).csv')
df_paid = pd.read_csv('paid_record(in).csv')

In [5]:
df_paid.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [6]:
df_paid['STATUS'].unique()

array(['X', '0', 'C', '1', '2', '3', '4', '5'], dtype=object)

In [7]:
df_main.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,DAYS_BIRTH_CLEAN,Letter,Main_ID
0,M,Y,Y,0.0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-4542.0,1.0,1.0,0.0,0.0,NaN,2.0,09/04/1990,L,5008804
1,M,Y,Y,0.0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-4542.0,1.0,1.0,0.0,0.0,NaN,2.0,09/04/1990,G,5008805
2,M,Y,Y,0.0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-1134.0,1.0,0.0,0.0,0.0,Security staff,2.0,06/05/1964,D,5008806
3,F,N,Y,0.0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,26/10/1970,I,5008808
4,F,NaN,Y,0.0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-3051.0,1.0,0.0,1.0,1.0,Sales staff,1.0,26/10/1970,Z,5008809


In [8]:
df_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438757 entries, 0 to 438756
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   New_ID   438757 non-null  object 
 1   ADDRESS  0 non-null       float64
dtypes: float64(1), object(1)
memory usage: 6.7+ MB


In [9]:
# arreglando  df_address
df_address = df_address.drop('ADDRESS', axis=1)
df_address[['id', 'address']] = df_address['New_ID'].str.split(',', n=1, expand=True)
df_address = df_address.drop('New_ID', axis=1)
df_address['Main_ID'] = df_address['id'].str.extract(r'(\d+)')
df_address = df_address.drop('id', axis=1)
df_address['Main_ID'] = df_address['Main_ID'].astype('Int64')
df_address['STATE'] = df_address['address'].str.extract(r'([A-Z]{2})')

In [10]:
# arreglando df_main
df_main['CNT_CHILDREN'] = df_main['CNT_CHILDREN'].replace('.','')
df_main['DAYS_EMPLOYED'] = df_main['DAYS_EMPLOYED'].replace('.','')
df_main['DAYS_EMPLOYED'] = df_main['DAYS_EMPLOYED'].abs()
df_main['AMT_INCOME_TOTAL'] = df_main['AMT_INCOME_TOTAL'].replace('.',',')


#Arreglando valores nulos
COLUMNS = ['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'DAYS_EMPLOYED', 'CNT_FAM_MEMBERS']
for col in COLUMNS:
    if df_main[col].mean() > 0:
        df_main[col] = df_main[col].replace({np.nan: df_main[col].mean()})
    else :   
        df_main[col] = df_main[col].replace(np.nan, 0)
COLUMNS = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE','FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL']
for col in COLUMNS:
    df_main[col] =  df_main[col].replace({np.nan: df_main[col].mode()[0]})
    
df_main['NAME_INCOME_TYPE'] = df_main['NAME_INCOME_TYPE'].fillna('SinInfo')
df_main['NAME_EDUCATION_TYPE'] = df_main['NAME_EDUCATION_TYPE'].fillna('other')
df_main['OCCUPATION_TYPE'] = df_main['OCCUPATION_TYPE'].fillna('other')

df_main['DAYS_BIRTH_CLEAN'] = pd.to_datetime(df_main['DAYS_BIRTH_CLEAN'], format='%d/%m/%Y')
df_main['DAYS_BIRTH_CLEAN'] = df_main['DAYS_BIRTH_CLEAN'].fillna(df_main['DAYS_BIRTH_CLEAN'].mean())
df_main['ANO'] = df_main['DAYS_BIRTH_CLEAN'].dt.year
df_main = df_main.drop('DAYS_BIRTH_CLEAN', axis=1)

In [11]:
# arreglando df_paid
df_paid.rename(columns={'ID': 'Main_ID'}, inplace=True)
df_paid = df_paid.groupby('Main_ID')['STATUS'].value_counts().unstack().reset_index().fillna(0)
df_paid['total'] = df_paid.iloc[:, 1:].sum(axis=1)


In [12]:
#categorizando a los pagadores en su ratio de pagos.

df_paid['class'] = np.nan
for lin in df_paid.index:
    total = df_paid.loc[lin, 'total']
    sum_c_0 = df_paid.loc[lin, 'C'] + df_paid.loc[lin, '0']
    perc_c_0 = (sum_c_0 / total) * 100 if total != 0 else 0
    
    if perc_c_0 >= 90:
        df_paid.at[lin, 'class'] = 'bom pagador'
    elif 70 <= perc_c_0 < 90 and df_paid.loc[lin, '3'] == 0 and df_paid.loc[lin, '4'] == 0 and df_paid.loc[lin, '5'] == 0:
        df_paid.at[lin, 'class'] = 'bom pagador'
    else:
        df_paid.at[lin, 'class'] = 'mau pagador'


C:\Users\User\AppData\Local\Temp\ipykernel_34124\328016011.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'bom pagador' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_paid.at[lin, 'class'] = 'bom pagador'


In [13]:
df_paid.head(20)

STATUS,Main_ID,0,1,2,3,4,5,C,X,total,class
0,5001711,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,bom pagador
1,5001712,10.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,19.0,bom pagador
2,5001713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,22.0,mau pagador
3,5001714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,mau pagador
4,5001715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,mau pagador
5,5001717,17.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,22.0,bom pagador
6,5001718,24.0,2.0,0.0,0.0,0.0,0.0,3.0,10.0,39.0,mau pagador
7,5001719,2.0,0.0,0.0,0.0,0.0,0.0,41.0,0.0,43.0,bom pagador
8,5001720,29.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,bom pagador
9,5001723,8.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,31.0,mau pagador


In [14]:
# Union de los dataframes
df_paid = df_paid [['Main_ID', 'class']]
df = pd.merge(df_main, df_address, on='Main_ID', how='inner')
df = df.merge(df_paid, how='inner', left_on='Main_ID', right_on='Main_ID')
df = df.dropna(how='all')
df = df.drop(columns=['address', 'Letter'])

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36650 entries, 0 to 36649
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CODE_GENDER          36650 non-null  object 
 1   FLAG_OWN_CAR         36650 non-null  object 
 2   FLAG_OWN_REALTY      36650 non-null  object 
 3   CNT_CHILDREN         36650 non-null  float64
 4   AMT_INCOME_TOTAL     36650 non-null  float64
 5   NAME_INCOME_TYPE     36650 non-null  object 
 6   NAME_EDUCATION_TYPE  36650 non-null  object 
 7   NAME_FAMILY_STATUS   36650 non-null  object 
 8   NAME_HOUSING_TYPE    36650 non-null  object 
 9   DAYS_EMPLOYED        36650 non-null  float64
 10  FLAG_MOBIL           36650 non-null  float64
 11  FLAG_WORK_PHONE      36650 non-null  float64
 12  FLAG_PHONE           36650 non-null  float64
 13  FLAG_EMAIL           36650 non-null  float64
 14  OCCUPATION_TYPE      36650 non-null  object 
 15  CNT_FAM_MEMBERS      36650 non-null 

In [16]:
df

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,Main_ID,ANO,STATE,class
0,M,Y,Y,0.0,427500.0,Working,Higher education,Civil marriage,Rented apartment,4542.0,1.0,1.0,0.0,0.0,other,2.0,5008804,1990,CT,bom pagador
1,F,N,Y,1.0,90000.0,State servant,Secondary / secondary special,Married,House / apartment,3659.0,1.0,0.0,0.0,0.0,Medicine staff,3.0,5008804,1984,CT,bom pagador
2,M,Y,Y,0.0,427500.0,Working,Higher education,Civil marriage,Rented apartment,4542.0,1.0,1.0,0.0,0.0,other,2.0,5008805,1990,AR,bom pagador
3,M,N,Y,0.0,180000.0,Working,Higher education,Civil marriage,With parents,390.0,1.0,0.0,0.0,0.0,Sales staff,2.0,5008805,1995,AR,bom pagador
4,M,Y,Y,0.0,112500.0,Working,Secondary / secondary special,Married,House / apartment,1134.0,1.0,0.0,0.0,0.0,Security staff,2.0,5008806,1964,AR,mau pagador
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36645,M,Y,Y,0.0,315000.0,Working,Secondary / secondary special,Married,House / apartment,2420.0,1.0,0.0,0.0,0.0,Managers,2.0,5149828,1979,DC,mau pagador
36646,F,N,Y,0.0,157500.0,Commercial associate,Higher education,Married,House / apartment,1325.0,1.0,0.0,1.0,1.0,Medicine staff,2.0,5149834,1989,AL,mau pagador
36647,F,N,Y,0.0,157500.0,Pensioner,Higher education,Married,House / apartment,1325.0,1.0,0.0,1.0,1.0,Medicine staff,2.0,5149838,1989,CO,mau pagador
36648,F,N,Y,0.0,283500.0,Working,Secondary / secondary special,Married,House / apartment,655.0,1.0,0.0,0.0,0.0,Sales staff,2.0,5150049,1973,CA,bom pagador


In [17]:
#Encoding
from sklearn.preprocessing import StandardScaler, LabelEncoder

scaler = StandardScaler()
df['AMT_INCOME_TOTAL'] = scaler.fit_transform(df[['AMT_INCOME_TOTAL']])
df['DAYS_EMPLOYED'] = scaler.fit_transform(df[['DAYS_EMPLOYED']])


for column in df.columns:
    if df[column].dtype == type(object):
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])


In [18]:
df.isnull().sum() * 100 / len(df)

CODE_GENDER            0.0
FLAG_OWN_CAR           0.0
FLAG_OWN_REALTY        0.0
CNT_CHILDREN           0.0
AMT_INCOME_TOTAL       0.0
NAME_INCOME_TYPE       0.0
NAME_EDUCATION_TYPE    0.0
NAME_FAMILY_STATUS     0.0
NAME_HOUSING_TYPE      0.0
DAYS_EMPLOYED          0.0
FLAG_MOBIL             0.0
FLAG_WORK_PHONE        0.0
FLAG_PHONE             0.0
FLAG_EMAIL             0.0
OCCUPATION_TYPE        0.0
CNT_FAM_MEMBERS        0.0
Main_ID                0.0
ANO                    0.0
STATE                  0.0
class                  0.0
dtype: float64

Acemos una prueba de modelos con Pycaret

In [19]:
data = df.sample(frac=0.8, random_state=342)
data

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,Main_ID,ANO,STATE,class
23045,1,1,0,0.0,-0.650348,0,4,3,5,-0.474401,1.0,0.0,0.0,0.0,8,1.0,5092453,1997,8,0
25375,0,1,1,2.0,-0.740183,5,1,1,1,-0.444714,1.0,0.0,0.0,0.0,0,4.0,5100382,1992,3,0
17891,0,1,0,0.0,0.831932,5,4,1,1,-0.448239,1.0,0.0,0.0,0.0,8,2.0,5068195,1974,12,0
14018,0,0,0,0.0,-1.548700,1,5,1,1,2.252579,1.0,0.0,0.0,0.0,18,2.0,5054382,1966,7,0
35313,1,1,1,0.0,3.077812,5,4,3,4,-0.435080,1.0,0.0,0.0,0.0,2,1.0,5148968,1983,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,1,1,1,0.0,0.831932,2,1,1,1,-0.469464,1.0,1.0,1.0,1.0,0,2.0,5008815,1976,3,0
28923,0,0,0,0.0,-0.425760,1,4,1,1,2.252579,1.0,0.0,1.0,0.0,18,2.0,5116598,1961,3,0
17347,1,1,1,1.0,0.382756,0,4,1,1,0.008881,1.0,0.0,0.0,0.0,8,3.0,5067509,1971,17,0
1673,0,1,1,2.0,0.831932,5,4,1,1,-0.444692,1.0,0.0,0.0,0.0,8,4.0,5010664,1986,6,0


In [120]:
data_unseen = df.drop(data.index)
data_unseen

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,Main_ID,ANO,STATE,class
5,0,0,0,0.0,-1.369030,1,4,1,1,2.252579,1.0,0.0,0.0,0.0,18,2.0,5008806,1966,2,1
14,0,0,1,0.0,0.966685,1,1,2,1,2.252579,1.0,0.0,0.0,0.0,18,1.0,5008812,1961,5,0
16,0,0,1,0.0,0.966685,1,1,2,1,2.252579,1.0,0.0,0.0,0.0,18,1.0,5008813,1961,6,0
19,0,0,1,0.0,0.158168,0,4,1,1,-0.462056,1.0,0.0,0.0,0.0,8,2.0,5008814,1983,4,0
25,0,0,1,0.0,-1.324112,5,4,1,1,-0.464251,1.0,1.0,1.0,0.0,2,2.0,5008819,1977,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36625,0,0,1,0.0,1.191273,1,1,1,1,0.008881,1.0,0.0,0.0,0.0,18,2.0,5145690,1964,9,1
36630,0,0,1,0.0,-0.515595,5,4,4,1,-0.468344,1.0,0.0,1.0,0.0,18,1.0,5146444,1983,9,1
36631,1,1,1,0.0,-0.066420,5,4,1,1,-0.464856,1.0,0.0,0.0,0.0,4,2.0,5148587,1992,3,1
36637,0,0,1,0.0,-0.066420,5,4,0,1,-0.448262,1.0,0.0,0.0,0.0,8,2.0,5149042,1969,4,1


In [121]:
data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)
print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (29320, 20)
Unseen Data For Predictions: (7330, 20)


In [122]:
from pycaret.classification import *
model_setup = setup(data = data, target = 'class', session_id=456)

,Description,Value
0,Session id,456
1,Target,class
2,Target type,Binary
3,Original data shape,"(29320, 20)"
4,Transformed data shape,"(29320, 20)"
5,Transformed train set shape,"(20524, 20)"
6,Transformed test set shape,"(8796, 20)"
7,Numeric features,19
8,Preprocess,True
9,Imputation type,simple


In [123]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [124]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.7561,0.7023,0.2945,0.5570,0.3849,0.2506,0.2705,0.1350
xgboost,Extreme Gradient Boosting,0.7505,0.6683,0.1825,0.5590,0.2751,0.1689,0.2078,0.0830
et,Extra Trees Classifier,0.7491,0.6894,0.3501,0.5245,0.4198,0.2678,0.2768,0.1000
lightgbm,Light Gradient Boosting Machine,0.7477,0.6590,0.0558,0.6638,0.1028,0.0653,0.1376,0.2060
gbc,Gradient Boosting Classifier,0.7415,0.5873,0.0066,0.7583,0.0130,0.0082,0.0509,0.2090
lr,Logistic Regression,0.7405,0.4812,0.0000,0.0000,0.0000,0.0000,0.0000,0.4490
svm,SVM - Linear Kernel,0.7405,0.4880,0.0000,0.0000,0.0000,0.0000,0.0000,0.0340
ridge,Ridge Classifier,0.7405,0.5242,0.0000,0.0000,0.0000,0.0000,0.0000,0.0070
ada,Ada Boost Classifier,0.7405,0.5420,0.0006,0.1167,0.0011,0.0005,0.0040,0.0630
lda,Linear Discriminant Analysis,0.7405,0.5242,0.0000,0.0000,0.0000,0.0000,0.0000,0.0090


In [126]:
model_rf = create_model('rf')
print(model_rf)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7623,0.6919,0.3152,0.5773,0.4078,0.2748,0.2945
1,0.7565,0.7051,0.3096,0.5556,0.3976,0.2601,0.2776
2,0.7579,0.7069,0.2795,0.5687,0.3748,0.2458,0.2696
3,0.7574,0.7042,0.2889,0.5641,0.3821,0.2503,0.2720
4,0.7593,0.7038,0.2801,0.5731,0.3763,0.2483,0.2728
5,0.7632,0.7206,0.3308,0.5752,0.4200,0.2846,0.3018
6,0.7471,0.6744,0.2726,0.5235,0.3585,0.2200,0.2382
7,0.7500,0.7127,0.2820,0.5338,0.3690,0.2312,0.2496
8,0.7519,0.7083,0.3083,0.5377,0.3919,0.2502,0.2655


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       monotonic_cst=None, n_estimators=100, n_jobs=-1,
                       oob_score=False, random_state=456, verbose=0,
                       warm_start=False)


In [127]:
evaluate_model(model_rf)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [128]:
#plot_model(model_rf, plot = 'feature')

In [129]:
#data_unseen

In [130]:
unseen_predictions = predict_model(model_rf, data=data_unseen)
unseen_predictions.head()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.7572,0.6955,0.3115,0.5693,0.4027,0.2663,0.2854


,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_EMPLOYED,...,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,Main_ID,ANO,STATE,class,prediction_label,prediction_score
0,0,0,0,0.0,-1.369030,1,4,1,1,2.252578,...,0.0,0.0,18,2.0,5008806,1966,2,1,0,0.79
1,0,0,1,0.0,0.966685,1,1,2,1,2.252578,...,0.0,0.0,18,1.0,5008812,1961,5,0,0,0.81
2,0,0,1,0.0,0.966685,1,1,2,1,2.252578,...,0.0,0.0,18,1.0,5008813,1961,6,0,0,0.74
3,0,0,1,0.0,0.158168,0,4,1,1,-0.462056,...,0.0,0.0,8,2.0,5008814,1983,4,0,0,0.78
4,0,0,1,0.0,-1.324112,5,4,1,1,-0.464251,...,1.0,0.0,2,2.0,5008819,1977,11,1,0,0.63


In [131]:
final_Model = finalize_model(model_rf)

In [132]:
#save_model(final_Model, 'Final_predict_Model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['CODE_GENDER', 'FLAG_OWN_CAR',
                                              'FLAG_OWN_REALTY', 'CNT_CHILDREN',
                                              'AMT_INCOME_TOTAL',
                                              'NAME_INCOME_TYPE',
                                              'NAME_EDUCATION_TYPE',
                                              'NAME_FAMILY_STATUS',
                                              'NAME_HOUSING_TYPE',
                                              'DAYS_EMPLOYED', 'FLAG_MOBIL',
                                              'FLAG_WORK_PHONE', 'FLAG_PHONE',
                                              'FLAG_EMAIL', 'OCCUPA...
                  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                         class_weight=None, criterion='gini',

Creando el modelo final

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


def train_model_forest(df):
    X = df.drop('class', axis=1)
    y = df['class']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=242)

    RFC = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='sqrt',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_samples_leaf=1,
                        min_samples_split=2, min_weight_fraction_leaf=0.0,
                        monotonic_cst=None, n_estimators=100, n_jobs=-1,
                        oob_score=False, random_state=456, verbose=0,
                        warm_start=False
    )
    RFC.fit(X_train, y_train)

    return RFC

In [27]:
modelo = train_model_forest(df)

In [28]:
y_preds = modelo.predict(X_test)

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.80      0.91      0.85      5463
           1       0.57      0.35      0.43      1867

    accuracy                           0.77      7330
   macro avg       0.68      0.63      0.64      7330
weighted avg       0.74      0.77      0.74      7330



In [31]:
import joblib
joblib.dump(modelo, 'modelo_train_prueba.pkl')

['modelo_train_prueba.pkl']